In [1]:
import os
import glob
import rasterio
from rasterio.io import MemoryFile
from rasterio.mask import mask
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping, box

In [ ]:


def calculate_percentage_covered(raster_paths, shapefile):
    # Read the shapefile
    gdf = shapefile.copy()
    
    # Initialize a dictionary to accumulate results
    coverage_dict = {idx: [] for idx in gdf.index}
    
    # Loop over each raster file path
    for raster_path in raster_paths:
        # Read the raster file
        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)  # Read the first band
            raster_meta = src.meta
            raster_bounds = src.bounds

        # Set all values < 1.0 to 0
        raster_data[raster_data < 1.0] = 0

        # Invert the binary values: 0 becomes 1 and 1 becomes 0
        inverted_raster_data = np.where(raster_data == 0, 1, 0)

        # Create an in-memory file for the modified raster data
        with MemoryFile() as memfile:
            with memfile.open(**raster_meta) as mem:
                mem.write(inverted_raster_data, 1)

                # Loop over each polygon and calculate the percentage of coverage
                for idx, row in gdf.iterrows():
                    polygon = row['geometry']
                    
                    # Check if the polygon is within the raster bounds
                    if polygon.intersects(box(*raster_bounds)):
                        # Mask the raster with the current polygon using the in-memory file
                        out_image, out_transform = mask(mem, [mapping(polygon)], crop=True)
                        # Flatten the array and remove masked values
                        masked_data = out_image[0].flatten()
                        masked_data = masked_data[masked_data != mem.nodata]
                        # Calculate the percentage of 1s
                        total_pixels = len(masked_data)
                        if total_pixels > 0:
                            percentage_ones = np.sum(masked_data == 1) / total_pixels * 100
                        else:
                            percentage_ones = 0
                        coverage_dict[idx].append(percentage_ones)
    
    # Combine the results from overlapping rasters
    for idx in coverage_dict:
        if coverage_dict[idx]:
            gdf.at[idx, 'shade_percent'] = np.mean(coverage_dict[idx])
        else:
            gdf.at[idx, 'shade_percent'] = 0

    return gdf

def find_raster_files(base_path, pattern):
    return glob.glob(os.path.join(base_path, '**', pattern), recursive=True)

# Example usage
base_path = '../data/clean_data/shade_comparison/buildings_trees'
pattern = 'Shadow_20240703_1300_LST.tif'  # Adjust the pattern as needed to match your files
raster_paths = find_raster_files(base_path, pattern)

# Assume verhardingen is a GeoDataFrame loaded earlier
# verhardingen = gpd.read_file('path_to_your_shapefile.shp')
coverage_results = calculate_percentage_covered(raster_paths, verhardingen)

# Print results
coverage_results.head()

In [ ]:


def calculate_percentage_covered(raster_paths, shapefile):
    # Read the shapefile
    gdf = shapefile.copy()
    
    # Initialize a dictionary to accumulate results
    coverage_dict = {idx: [] for idx in gdf.index}
    
    # Loop over each raster file path
    for raster_path in raster_paths:
        # Read the raster file
        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)  # Read the first band
            raster_meta = src.meta
            raster_bounds = src.bounds

        # Set all values < 1.0 to 0
        raster_data[raster_data < 1.0] = 0

        # Invert the binary values: 0 becomes 1 and 1 becomes 0
        inverted_raster_data = np.where(raster_data == 0, 1, 0)

        # Create an in-memory file for the modified raster data
        with MemoryFile() as memfile:
            with memfile.open(**raster_meta) as mem:
                mem.write(inverted_raster_data, 1)

                # Loop over each polygon and calculate the percentage of coverage
                for idx, row in gdf.iterrows():
                    polygon = row['geometry']
                    
                    # Check if the polygon is within the raster bounds
                    if polygon.intersects(box(*raster_bounds)):
                        # Mask the raster with the current polygon using the in-memory file
                        out_image, out_transform = mask(mem, [mapping(polygon)], crop=True)
                        # Flatten the array and remove masked values
                        masked_data = out_image[0].flatten()
                        masked_data = masked_data[masked_data != mem.nodata]
                        # Calculate the percentage of 1s
                        total_pixels = len(masked_data)
                        if total_pixels > 0:
                            percentage_ones = np.sum(masked_data == 1) / total_pixels * 100
                        else:
                            percentage_ones = 0
                        coverage_dict[idx].append(percentage_ones)
    
    # Combine the results from overlapping rasters
    for idx in coverage_dict:
        if coverage_dict[idx]:
            gdf.at[idx, 'shade_percent'] = np.mean(coverage_dict[idx])
        else:
            gdf.at[idx, 'shade_percent'] = 0

    return gdf

def find_raster_files(base_path, pattern):
    return glob.glob(os.path.join(base_path, '**', f'*{pattern}'), recursive=True)

# Example usage
base_path = '../results/output/271110/building_shade/'
pattern = '20240620_1500_LST.tif'  # Adjust the pattern as needed to match your files
raster_paths = find_raster_files(base_path, pattern)

print(f' Found {len(raster_paths)} raster files')

# Assume verhardingen is a GeoDataFrame loaded earlier
verhardingen = gpd.read_file('../data/raw_data/AMS/Q1_20230126_ingekort.gpkg')
coverage_results = calculate_percentage_covered(raster_paths, verhardingen)

# Print results
coverage_results.head()